<h1><center>Создание сетей друзей ВКонтакте</center></h1>

In [10]:
with open('token.txt', 'w') as file:
    file.write('...')

In [14]:
# загрузка библиотек
import requests
import numpy as np
from datetime import datetime
from time import sleep
import pandas as pd
import tqdm
import re
import math
import warnings
warnings.filterwarnings("ignore")

# загрузка токена, id и уточнение версии
with open('id.txt') as file:
    myid = file.read()

# версия используемого API
version = '5.131'
# подгружаем токен из файла на компьютере
with open('token.txt') as file:
    token = file.read()

<h2><center>Сбор данных с помощью VK API</center></h2>

In [15]:
def get_id_of_user_from_link(user_link, method='util.resolveScreeenName'):
    '''
    Функция get_id_of_user_from_link(user_link) принимает на вход ссылку на страницу пользователя ВКонтакте
    и возвращает его id.
    
    Args:
        :user_link: string - ссылка на пользователя;
    Returns:
        :user_id: string - id пользователя в виде строки
    Example:
        >>>> one_user_id = get_id_of_user_from_link('https://vk.com/lika.kapustina')
        >>>> one_user_id # 441721976
    '''

    screen_name = user_link.replace('https://vk.com/', '')

    # может быть такое, что ссылка на пользователя содержит id (vk.com/id777)
    if screen_name.startswith('id') == True and screen_name[2:].isdigit() == True:
        user_id = screen_name.replace('id', '')

    # если нет, нам нужно сделать запрос к ВК и получить id пользователя
    else:
        method_for_get_id = 'utils.resolveScreenName'
        parameters='screen_name=' + screen_name
        url = 'https://api.vk.com/method/' + method_for_get_id +'?'+ parameters + '&v=' + version + '&access_token=' + token
        response = requests.get(url)
        response = response.json()
        user_id = response['response']['object_id']
        
    # возвращаю id
    return user_id

# пример
# one_user_id = get_id_of_user_from_link('https://vk.com/lika.kapustina')
# print(one_user_id)
# one_user_id = get_id_of_user_from_link('https://vk.com/id121096')
# print(one_user_id)

def get_id_of_all_users_links(users_links, method='util.resolveScreeenName'):
    '''
    Функция get_ids_of_all_users_from_links(user_links) принимает на вход СПИСОК со ссылками на аккаунты пользователей,
    обращается внутри себя к методу `get_id_of_user_from_link(user_link)`,
    и возвращает СТРОКУ с id пользователей, записанных через строку.
    
    Args:
        :users_links: list - список с ссылками на пользователей;
    Returns:
        :users_ids_string: string - id пользователей, записанные через запятую в виде строки.
    Example:
        >>>> list_with_ids = get_id_of_all_users_links(['https://vk.com/lika.kapustina','https://vk.com/sdilov','https://vk.com/sofiagreseva'])
        >>>> list_with_ids
        
    '''
    users_ids_list = [] # создаем пустой список
    
    # идем по всему списку из ссылок
    for user_link in tqdm.tqdm(users_links):
        user_id = get_id_of_user_from_link(f'{user_link}') # получаем id пользователя
        users_ids_list.append(user_id)
        sleep(0.15) # небольшая пауза    
    
    users_ids_string = ','.join([str(i) for i in users_ids_list])
    
    return users_ids_string # возвращаем строку с id пользователей

# пример
# list_with_ids = get_id_of_all_users_links(['https://vk.com/lika.kapustina',
#                                           'https://vk.com/sdilov',
#                                           'https://vk.com/sofiagreseva'])
# list_with_ids

def get_friends_of_user_only_ids(user_id, method='friends.get'):
    '''Функция get_friends_of_user_only_ids принимает на вход id пользователя ВКонтакте и возвращает строку, где через
    запятую перечислены id его друзей.
    
    Method: https://dev.vk.com/method/friends.get
    
    Args:
        :user_id: id пользователя ВКонтакте
    Returns:
        :friends_string: string с id друзей пользователя
    Example:
        >>>> limarenko_friends = get_friends_of_user('533710525')
        >>>> limarenko_friends # '626130,957188,1053362,2058801,3279648,4120747,5823842,8523846,11146055,11860274,25758390,169272036,171462727,182026369,196603818,322659228,462979979,520753933,538671248,550960397,550966564,551354669,551382456,551506228,578750417,606932606,616844912,633974128,660930698,665881497,670425420,689458339,703017057,703621379,708192084,708405503,710516910,710520153,710787650,711209497,711761122,711978083,711991565,712541914,712544833,712545388,713904031

    '''
    # осуществляем запрос
    parameters = 'count=5000' # берем максимум - 5000 друзей
    url = 'https://api.vk.com/method/' + method +'?' + parameters + '&v=' + version + '&access_token=' + token + '&user_id=' + user_id
    response = requests.get(url)
    one_data = response.text
    
    # обрабатываем данные
    first_index = one_data.index('[')
    second_index = one_data.index(']')
    friends_string = one_data[first_index+1:second_index]
    return friends_string # возвращаем строку с id друзей пользователя

# пример
# limarenko_friends = get_friends_of_user_only_ids('533710525')
# limarenko_friends

def get_friends_of_user_full_info_in_df(user_id, type_of = 'id', method='friends.get'):
    '''Функция get_friends_of_user_info_df принимает на вход ИЛИ id пользователя ИЛИ ССЫЛКУ ВКонтакте 
    и возвращает датафрейм, где перечислены данные его друзей.
    
    Method: https://dev.vk.com/method/friends.get
    
    Args:
        :user_id: id пользователя ВКонтакте;
        :type_of: тип введенных данных (id пользователя ('id')/ссылка на него('link')).
        
    Returns:
        :pandas.DataFrame: датафрейм с информацией о друзьях пользователя.
        
        > friend_owner: id пользователя, информация о чьих друзьях указана далее.
        > user_id: id пользователя;
        > user_name: имя пользователя;
        > user_surname: фамилия пользователя;
        > user_sex: пол пользователя;
        > user_domain: короткое имя пользователя (нужно для ссылки);
        > user_bdate: дата рождения пользователя (иногда указывается без года);
        > user_city: город пользователя;
        > user_country: страна пользоваателя;
        > user_is_closed: закрыта ли страница пользователя;
        > user_link: сссылка на страницу пользователя.
        
    Example:
        >>>> limarenko_friends = get_friends_of_user('533710525')
        >>>> limarenko_friends # '626130,957188,1053362,2058801,3279648,4120747,5823842,8523846,11146055,11860274,25758390,169272036,171462727,182026369,196603818,322659228,462979979,520753933,538671248,550960397,550966564,551354669,551382456,551506228,578750417,606932606,616844912,633974128,660930698,665881497,670425420,689458339,703017057,703621379,708192084,708405503,710516910,710520153,710787650,711209497,711761122,711978083,711991565,712541914,712544833,712545388,713904031

    '''
    
    # если type_of = 'link' (то есть, не id), нужно получить id пользователя
    if type_of != 'id':
        link = user_id #значит, нам попалась ссылка и из нее нужно вынуть screen_name пользователя
        screen_name = link.replace('https://vk.com/', '')
        
        # может быть такое, что ссылка на полььзователя содержит id (vk.com/id777)
        if screen_name.startswith('id') == True and screen_name[2:].isdigit() == True:
            user_id = screen_name.replace('id', '')
        
        # если нет, нам нужно сделать запрос к ВК и получить id пользователя
        else:
            method_for_get_id = 'utils.resolveScreenName'
            parameters='screen_name=' + screen_name
            url = 'https://api.vk.com/method/' + method_for_get_id +'?'+ parameters + '&v=' + version + '&access_token=' + token
            response = requests.get(url)
            response = response.json()
            user_id = response['response']['object_id']

    
    # осуществляем запрос
    parameters = 'count=5000' + '&extended=1' + '&fields=bdate,city,country,sex,domain' + '&name_case=nom' # берем максимум - 5000 друзей
    url = 'https://api.vk.com/method/' + method +'?' + parameters + '&v=' + version + '&access_token=' + token + '&user_id=' + str(user_id)
    response = requests.get(url)
    response = response.json()
    
    # собираем данные для датафрейма
    number_of_friends = response['response']['count'] # число друзей у пользователя
    
    # список: чьи друзья в этом датафрейме?
    owner_of_friends = [user_id] * number_of_friends
    
    
    # общая информация, получается без необходимости использовать try-except.
    friends_ids = [response['response']['items'][i]['id'] for i in range(number_of_friends)]
    friends_first_names = [response['response']['items'][i]['first_name'] for i in range(number_of_friends)]
    friends_second_names = [response['response']['items'][i]['last_name'] for i in range(number_of_friends)]
    friends_sexs = [response['response']['items'][i]['sex'] for i in range(number_of_friends)]
    friends_is_closed = [response['response']['items'][i]['is_closed'] for i in range(number_of_friends)]


    
    # информация, которую нужно получать через try-except
    friends_domains = []
    for i in range(number_of_friends):
        try:
            one_domain = response['response']['items'][i]['domain']
        except:
            one_domain = str(response['response']['items'][i]['id'])
        friends_domains.append(one_domain)
        
        
    ## ссылки на пользвателей; если есть domain - значит, с domain; если нет - значит, с id
    friends_links = list(map(lambda x: f'https://vk.com/{x}' if x.isdigit() == False else \
                       f'https://vk.com/id{x}', friends_domains))

    friends_bdates = []
    for i in range(number_of_friends):
        try:
            one_bdates = response['response']['items'][i]['bdate']
        except:
            one_bdates = ''
        friends_bdates.append(one_bdates)


    friends_cities = []
    for i in range(number_of_friends):
        try:
            one_city = response['response']['items'][i]['city']['title']
        except:
            one_city = ''
        friends_cities.append(one_city)


    friends_countries = []
    for i in range(number_of_friends):
        try:
            one_country = response['response']['items'][i]['country']['title']
        except:
            one_country = ''
        friends_countries.append(one_country)
        
        
    # собираем датафрейм
    friends_df = pd.DataFrame({'friend_owner': owner_of_friends,
                               'user_id': friends_ids,
                              'user_name': friends_first_names,
                              'user_surname': friends_second_names,
                              'user_sex': friends_sexs,
                              'user_domain': friends_domains,
                              'user_bdate': friends_bdates,
                              'user_city': friends_cities,
                              'user_country': friends_countries,
                              'user_is_closed': friends_is_closed,
                              'user_link': friends_links})
    return friends_df

# пример 1
# friends_data = get_friends_of_user_full_info_in_df('533710525')
# friends_data

# пример 2
# friends_data = get_friends_of_user_full_info_in_df('https://vk.com/lika.kapustina', type_of = 'link')
# friends_data

def get_friends_of_a_few_users_full_info_in_df(users_ids, type_of='id', method='friends.get'):
    '''Функция get_friends_of_a_few_users_info_df принимает на вход строку с id пользователей ВКонтакте,
    записанными через запятую, обращается к функции get_friends_of_user_full_info_in_df(user_id),
    и возвращает pandas.DataFrame с информацией о всех друзьях всех указанных пользователей
    
    Method: https://dev.vk.com/method/friends.get
    
    Args:
        :users_ids: string: ИЛИ id пользователей ВКонтакте ИЛИ ссылки на страницы пользователей ВКонтакте, записанных через запятую;
        :type_of: string: тип данных на входе (по умолчанию 'id' - id пользователей), либо 'link' - ссылки на пользователей.
    
    Returns:
        :pandas.DataFrame: датафрейм с информацией о друзьях пользователей, чьи id переданы функции.
        > friend_owner: id пользователя, информация о чьих друзьях указана далее.
        > user_id: id пользователя;
        > user_name: имя пользователя;
        > user_surname: фамилия пользователя;
        > user_sex: пол пользователя;
        > user_domain: короткое имя пользователя (нужно для ссылки);
        > user_bdate: дата рождения пользователя (иногда указывается без года);
        > user_city: город пользователя;
        > user_country: страна пользоваателя;
        > user_is_closed: закрыта ли страница пользователя;
        > user_link: сссылка на страницу пользователя.
        
    Example:
        >>>> some_friends = get_friends_of_a_few_users_full_info_in_df('48905537,12900237')
        >>>> some_friends
        
    Exmple:
        >>>> my_friends_list = ['https://vk.com/zzimablue',
                                   'https://vk.com/ksperov',
                                   'https://vk.com/sofiagreseva',
                                   'https://vk.com/sdilov',
                                   'https://vk.com/narepapoyan',
                                   'https://vk.com/kirillmuzyka']
        >>>> my_friends_string = ','.join(my_friends_list)
        >>>> friends_of_my_friends = get_friends_of_a_few_users_full_info_in_df(my_friends_string, type_of='link')
        >>>> friends_of_my_friends
        
        
    '''
    
    users_ids = users_ids.split(',')
    
    full_users_friends_data = pd.DataFrame()
    
    for user_id in tqdm.tqdm(users_ids):
        try:
            one_user_friends_data = get_friends_of_user_full_info_in_df(user_id,
                                                                       type_of = type_of)
            full_users_friends_data = pd.concat([full_users_friends_data, one_user_friends_data])
            sleep(0.4)
        except Exception as e:
            print(e)
    
    return full_users_friends_data

# ПРИМЕР 1
# my_friends_list = ['https://vk.com/zzimablue', 'https://vk.com/ksperov', 'https://vk.com/sofiagreseva', 'https://vk.com/sdilov', 'https://vk.com/narepapoyan', 'https://vk.com/kirillmuzyka']
# my_friends_string = ','.join(my_friends_list)
# friends_of_my_friends = get_friends_of_a_few_users_full_info_in_df(my_friends_string, type_of='link')
# friends_of_my_friends

# ПРИМЕР 2
# some_friends = get_friends_of_a_few_users_full_info_in_df('48905537,12900237')
# some_friends

In [16]:
friends_data = get_friends_of_user_full_info_in_df('https://vk.com/vavaschenko', type_of = 'link')
friends_data

,friend_owner,user_id,user_name,user_surname,user_sex,user_domain,user_bdate,user_city,user_country,user_is_closed,user_link
0,234851997,9804,Никита,Бешта,2,beshta,25.8.1988,,,True,https://vk.com/beshta
1,234851997,68046,Руслан,Хаиткулов,2,paginatorx,7.12,Москва,Россия,False,https://vk.com/paginatorx
2,234851997,1164920,Игорь,Томашов,2,itomashov,23.9.1988,Москва,Россия,False,https://vk.com/itomashov
3,234851997,1320605,Ольга,Савинская,1,id1320605,,Москва,Россия,False,https://vk.com/id1320605
4,234851997,1441323,Олеся,Ващенко,1,id1441323,6.5,Москва,Россия,False,https://vk.com/id1441323
...,...,...,...,...,...,...,...,...,...,...,...
394,234851997,667598056,Anastasia,Afonina,1,kkktasy,1.5,Москва,Россия,True,https://vk.com/kkktasy
395,234851997,672578683,Степан,Беспалов,2,id672578683,25.12.2004,,,True,https://vk.com/id672578683
396,234851997,673660821,Андрей,Полищук,2,hot_frosty_woman,19.3.2004,Мирный,Россия,False,https://vk.com/hot_frosty_woman
397,234851997,686540851,Владислав,Фурсов,2,id686540851,,,,False,https://vk.com/id686540851


In [18]:
my_friends_list = friends_data['user_link'].tolist()
my_friends_string = ','.join(my_friends_list)
friends_of_my_friends = get_friends_of_a_few_users_full_info_in_df(my_friends_string, type_of='link')
friends_of_my_friends

  0%|                                           | 1/399 [00:00<03:57,  1.67it/s]

'response'


  5%|█▉                                        | 18/399 [00:43<11:32,  1.82s/it]

'response'


  6%|██▎                                       | 22/399 [00:51<12:00,  1.91s/it]

...

'response'


 99%|████████████████████████████████████████▋| 396/399 [09:49<00:01,  1.72it/s]

'response'


100%|█████████████████████████████████████████| 399/399 [09:52<00:00,  1.49s/it]


,friend_owner,user_id,user_name,user_surname,user_sex,user_domain,user_bdate,user_city,user_country,user_is_closed,user_link
0,68046,2841,Юлия,Кузнецова,1,id2841,25.3.1987,Москва,Россия,False,https://vk.com/id2841
1,68046,10822,Андрей,Алдашев,2,id10822,28.3.1983,Москва,Россия,False,https://vk.com/id10822
2,68046,14024,Алия,Пашкеева,1,id14024,12.7.1988,Москва,Россия,False,https://vk.com/id14024
3,68046,15233,Наталия,Наймарк,1,nataliya_mogilevskaya,22.11,,,False,https://vk.com/nataliya_mogilevskaya
4,68046,15357,Ким,Галачян,2,kimezz,11.8.1988,Москва,Россия,False,https://vk.com/kimezz
...,...,...,...,...,...,...,...,...,...,...,...
1287,786286130,785443059,Амина,Гончаренко,1,id785443059,22.3.2004,,,False,https://vk.com/id785443059
1288,786286130,793417890,София,Ситнова,1,id793417890,17.5.1972,,,False,https://vk.com/id793417890
1289,786286130,793556776,Роман,Семенов,2,id793556776,6.7.2004,,,False,https://vk.com/id793556776
1290,786286130,793837037,Kevin,Calderon,2,id793837037,21.2.2004,México,Мексика,False,https://vk.com/id793837037


<h2><center>Построение сети</center></h2>

In [83]:
friends_data

,friend_owner,user_id,user_name,user_surname,user_sex,user_domain,user_bdate,user_city,user_country,user_is_closed,user_link
0,441721976,130896,Егор,Юрескул,2,sydney,29.9.1988,Москва,Россия,False,https://vk.com/sydney
1,441721976,709743,Мария,Быстрова,1,mariabystrova,9.9,Москва,Россия,False,https://vk.com/mariabystrova
2,441721976,2416234,Zemtsov,Dmitry,2,dmitry.zemtsov,9.8.1984,Москва,Россия,False,https://vk.com/dmitry.zemtsov
3,441721976,4144285,Марта,Измайлова,1,id4144285,29.5,Москва,Россия,False,https://vk.com/id4144285
4,441721976,4762997,Никита,Власов,2,id4762997,3.3.1985,Москва,Россия,False,https://vk.com/id4762997
...,...,...,...,...,...,...,...,...,...,...,...
577,441721976,671432786,Sukhayl,Niyazov,2,fallibilist,,Edinburgh,Великобритания,False,https://vk.com/fallibilist
578,441721976,689200024,James,Penrose,2,id689200024,4.2.2002,,,False,https://vk.com/id689200024
579,441721976,690288246,Саша,Пожарская,1,iknoweveryonehasdreams,22.5.2001,,,False,https://vk.com/iknoweveryonehasdreams
580,441721976,709104191,Саша,Волкова,1,y4vk_pv,10.9.2002,,,False,https://vk.com/y4vk_pv


In [19]:
my_friends_ids = friends_data['user_id'].tolist()
print(len(my_friends_ids)) # сколько всего друзей
print(my_friends_ids[0:5]) # пример

399
[9804, 68046, 1164920, 1320605, 1441323]


In [20]:
my_friends_ids.append(friends_data['friend_owner'].unique()[0]) # добавляем в список себя

In [21]:
# теперь берем из полного списка только тех чье owner id есть в my_friends_ids
# и те чей user_id есть в списке с id "моих" друзей
friends_of_my_friends[(friends_of_my_friends['friend_owner'].isin(my_friends_ids)) &\
                     friends_of_my_friends['user_id'].isin(my_friends_ids)]

,friend_owner,user_id,user_name,user_surname,user_sex,user_domain,user_bdate,user_city,user_country,user_is_closed,user_link
214,68046,1164920,Игорь,Томашов,2,itomashov,23.9.1988,Москва,Россия,False,https://vk.com/itomashov
317,68046,2881398,Николай,Саперов,2,nikolaysaperov,30.6.1984,Москва,Россия,False,https://vk.com/nikolaysaperov
347,68046,3754814,Юлия,Апанасенко,1,julia_apanasenko,18.11,Москва,Россия,False,https://vk.com/julia_apanasenko
446,68046,8761370,Дмитрий,Турчанинов,2,turchaninovd,9.6,Калининград,Россия,False,https://vk.com/turchaninovd
490,68046,12247792,Соня,Анисимова,1,id12247792,23.8,Москва,Россия,False,https://vk.com/id12247792
...,...,...,...,...,...,...,...,...,...,...,...
1098,786286130,561708614,Максим,Мицкевич,2,mitskm,30.3.2004,Москва,Россия,False,https://vk.com/mitskm
1099,786286130,562095655,Daniel,Makarenko,2,sssshaggy,24.6,Москва,Россия,False,https://vk.com/sssshaggy
1101,786286130,562888592,Владислав,Кудинов,2,a1122,21.4,Москва,Россия,False,https://vk.com/a1122
1105,786286130,563489541,Дмитрий,Корнилов,2,id563489541,16.6.2005,,,False,https://vk.com/id563489541


In [22]:
small_df = friends_of_my_friends[['user_id', 'user_name', 'user_surname']].drop_duplicates()
small_df

,user_id,user_name,user_surname
0,2841,Юлия,Кузнецова
1,10822,Андрей,Алдашев
2,14024,Алия,Пашкеева
3,15233,Наталия,Наймарк
4,15357,Ким,Галачян
...,...,...,...
1281,752618085,Родион,Сапрыкин
1283,764150541,Ника,Горбунова
1286,782531014,Тоня,Вальтц
1289,793556776,Роман,Семенов


In [23]:
friends_of_my_friends = pd.merge(friends_of_my_friends, small_df, left_on='friend_owner', right_on='user_id')

In [24]:
friends_of_my_friends = friends_of_my_friends[['user_id_x', 'user_name_x', 'user_surname_x',
                      'user_id_y', 'user_name_y', 'user_surname_y']]
friends_of_my_friends['user_x'] = friends_of_my_friends['user_name_x'] + ' ' + friends_of_my_friends['user_surname_x']
friends_of_my_friends['user_y'] = friends_of_my_friends['user_name_y'] + ' ' + friends_of_my_friends['user_surname_y']
friends_of_my_friends

,user_id_x,user_name_x,user_surname_x,user_id_y,user_name_y,user_surname_y,user_x,user_y
0,2841,Юлия,Кузнецова,68046,Руслан,Хаиткулов,Юлия Кузнецова,Руслан Хаиткулов
1,10822,Андрей,Алдашев,68046,Руслан,Хаиткулов,Андрей Алдашев,Руслан Хаиткулов
2,14024,Алия,Пашкеева,68046,Руслан,Хаиткулов,Алия Пашкеева,Руслан Хаиткулов
3,15233,Наталия,Наймарк,68046,Руслан,Хаиткулов,Наталия Наймарк,Руслан Хаиткулов
4,15357,Ким,Галачян,68046,Руслан,Хаиткулов,Ким Галачян,Руслан Хаиткулов
...,...,...,...,...,...,...,...,...
182272,785443059,Амина,Гончаренко,786286130,Анна,Соловьева,Амина Гончаренко,Анна Соловьева
182273,793417890,София,Ситнова,786286130,Анна,Соловьева,София Ситнова,Анна Соловьева
182274,793556776,Роман,Семенов,786286130,Анна,Соловьева,Роман Семенов,Анна Соловьева
182275,793837037,Kevin,Calderon,786286130,Анна,Соловьева,Kevin Calderon,Анна Соловьева


In [25]:
friends_of_my_friends = friends_of_my_friends[(friends_of_my_friends['user_id_x'].isin(my_friends_ids)) &\
                     friends_of_my_friends['user_id_y'].isin(my_friends_ids)]
friends_of_my_friends

,user_id_x,user_name_x,user_surname_x,user_id_y,user_name_y,user_surname_y,user_x,user_y
214,1164920,Игорь,Томашов,68046,Руслан,Хаиткулов,Игорь Томашов,Руслан Хаиткулов
317,2881398,Николай,Саперов,68046,Руслан,Хаиткулов,Николай Саперов,Руслан Хаиткулов
347,3754814,Юлия,Апанасенко,68046,Руслан,Хаиткулов,Юлия Апанасенко,Руслан Хаиткулов
446,8761370,Дмитрий,Турчанинов,68046,Руслан,Хаиткулов,Дмитрий Турчанинов,Руслан Хаиткулов
490,12247792,Соня,Анисимова,68046,Руслан,Хаиткулов,Соня Анисимова,Руслан Хаиткулов
...,...,...,...,...,...,...,...,...
182083,561708614,Максим,Мицкевич,786286130,Анна,Соловьева,Максим Мицкевич,Анна Соловьева
182084,562095655,Daniel,Makarenko,786286130,Анна,Соловьева,Daniel Makarenko,Анна Соловьева
182086,562888592,Владислав,Кудинов,786286130,Анна,Соловьева,Владислав Кудинов,Анна Соловьева
182090,563489541,Дмитрий,Корнилов,786286130,Анна,Соловьева,Дмитрий Корнилов,Анна Соловьева


In [ ]:
import networkx as nx
G = nx.from_pandas_edgelist(df, 'user_x', 'user_y')
nx.write_gexf(G, "friends_network.gexf")

In [ ]:
# и этот файл friends_network.gexf нужно загрузить в Gephi lite и визуализировать!